In [11]:
from langgraph.graph import StateGraph,START,END
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import TypedDict

In [12]:
llm = ChatOllama(
    model = 'qwen2:7b',
    temperature=0.7
)

In [13]:
class LLMstate(TypedDict):
    questions:str
    answer:str

In [14]:
def llm_query(state: LLMstate) -> LLMstate:
    que = state['questions']

    prompt = PromptTemplate(
        template='answer the question: {topic}',
        input_variables=['topic']
    )

    parser = StrOutputParser()

    chain = prompt | llm | parser

    # Execute the chain
    answer = chain.invoke({"topic": que})

    state['answer'] = answer

    return state

In [15]:
graph = StateGraph(LLMstate)

graph.add_node('llm_query',llm_query)

graph.add_edge(START,'llm_query')
graph.add_edge('llm_query',END)

workflow=graph.compile()

In [16]:
initial_state = {'questions':'weight of 1 liter water'}

final_state=workflow.invoke(initial_state)

print(final_state)

{'questions': 'weight of 1 liter water', 'answer': 'The weight of 1 liter of water at standard conditions (temperature of 4 degrees Celsius and atmospheric pressure) is approximately 1 kilogram. This relationship between volume and mass—specifically that 1 liter of water has a mass close to 1 kilogram—is due to the density of water, which is about 1 gram per milliliter or 1000 kilograms per cubic meter at this temperature.'}
